# Crear SparkContext y SparkSession

In [1]:
from pyspark import SparkContext
sc = SparkContext(master = 'local')

from pyspark.sql import SparkSession
spark = SparkSession.builder \
          .appName("Python Spark SQL basic example") \
          .config("spark.some.config.option", "some-value") \
          .getOrCreate()

# TF, IDF y TF-IDF

* TF es la abreviatura de **Frecuencia de Término**. Es simplemente la frecuencia de un término en un documento. Cuanto más alta sea la TF para un término específico, más importante será ese término para ese documento.

* IDF es la abreviatura de **Inverse Document Frequency**. Es la frecuencia de los documentos que contienen un término específico. Si existe un término en cada documento, entonces la Frecuencia de documentos es la mayor y es 1. Y la Frecuencia inversa de documentos será la menor. En esta situación, este término no es informativo para clasificar los documentos. Cuanto más alto es el IDF, más relavante es el término.

* TF-IDF es el producto de TF e IDF. Se obtiene un TF-IDF alto cuando el Término Frecuencia es alto y la Frecuencia de Documento es baja (IDF es alta).


# Término Frecuencia, HashingTF y CountVectorizer

Pyspark tiene dos funciones para calcular frecuencias de términos a partir de documentos: el **`HashingTF()`** y el **`CountVectorizer()`**. Estas dos funciones hacen dos cosas:

1. Indexación de términos: conversión de palabras a números.
2. Calcule las frecuencias de los plazos para cada documento.

El `HashingTF()` utiliza la función Murmurhash 3 para mapear una característica sin procesar (un término) en un índice (un número). El hashing es el proceso de transformar datos de tamaño arbitrario en datos de tamaño fijo, generalmente más cortos. Las frecuencias de término se calculan en base a los índices generados. Para el método HashingTF(), el proceso de mapeo es muy barato. Porque cada mapeo de término a índice es independiente de otros mapeos de término a índice. La función de hashing toma una entrada única y genera un "resultado único". Sin embargo, puede ocurrir una colisión**, lo que significa que diferentes características (términos) pueden ser ajustados al mismo índice.

El **`CountVectorizer()`** indexa los términos por orden descendente de frecuencias de términos en todo el corpus, NO las frecuencias de términos en el documento. Después del proceso de indexación, las frecuencias de los términos se calculan mediante documentos.

## Crear algunos datos

In [2]:
import pandas as pd
pdf = pd.DataFrame({
        'terms': [
            ['spark', 'spark', 'spark', 'is', 'awesome', 'awesome'],
            ['I', 'love', 'spark', 'very', 'very', 'much'],
            ['everyone', 'should', 'use', 'spark']
        ]
    })
df = spark.createDataFrame(pdf)
df.show(truncate=False)

+-------------------------------------------+
|terms                                      |
+-------------------------------------------+
|[spark, spark, spark, is, awesome, awesome]|
|[I, love, spark, very, very, much]         |
|[everyone, should, use, spark]             |
+-------------------------------------------+



## HashingTF

El **numFeatures** toma un número entero, que debe ser mayor que el número total de términos en el corpus. Y debería ser una potencia de dos para que las características se asignen uniformemente a las columnas.

In [3]:
from pyspark.ml.feature import HashingTF
from pyspark.ml import Pipeline

hashtf = HashingTF(numFeatures=pow(2, 4), inputCol='terms', outputCol='features(numFeatures), [index], [term frequency]')
stages = [hashtf]
pipeline = Pipeline(stages=stages)

In [4]:
pipeline.fit(df).transform(df).show(truncate=False)

+-------------------------------------------+------------------------------------------------+
|terms                                      |features(numFeatures), [index], [term frequency]|
+-------------------------------------------+------------------------------------------------+
|[spark, spark, spark, is, awesome, awesome]|(16,[1,15],[4.0,2.0])                           |
|[I, love, spark, very, very, much]         |(16,[0,1,2,8,12],[1.0,1.0,1.0,2.0,1.0])         |
|[everyone, should, use, spark]             |(16,[1,9,13],[2.0,1.0,1.0])                     |
+-------------------------------------------+------------------------------------------------+



Puede notar que el primer documento tiene tres términos distintos, pero sólo se obtienen dos frecuencias de términos. Esta aparente discrepancia se debe a una colisión **hashing**: tanto `spark` como `is` se están apresurando a `1`. La frecuencia del término para el índice `1` en el primer documento es `4.0` correspondiente a los tres conteos de `spark` y el conteo de `is`. La probabilidad de una colisión de hashing puede reducirse aumentando el parámetro `numFeatures` pasado a la función `HashingTF` (el valor por defecto es por ejemplo $2^18 = 262,144$).

## CountVectorizer

La función **`CountVectorizer()`** tiene tres parámetros para controlar qué términos se mantendrán como características.

* minTF: se eliminarán las características que tengan una frecuencia de término inferior a minTF. Si minTF=1minTF=1, no se eliminará ninguna característica.
* minDF: se eliminarán las características que tengan una frecuencia de documentos inferior a minDF. Si minDF=1minDF=1, no se eliminará ninguna característica.
* vocabSize: mantiene los términos de las frecuencias superiores de vocabSize.

En el ejemplo siguiente, el `minTF=1.0,minDF=1.0minTF=1.0,minDF=1.0` y `vocabSize=20vocabSize=20`, que es mayor que el número total de términos. Por lo tanto, todas las características (términos) se mantendrán.

In [5]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml import Pipeline

countvectorizer = CountVectorizer(minTF=1.0, minDF=1.0, vocabSize=20, 
                                  inputCol='terms', outputCol='features(vocabSize), [index], [term frequency]')
stages = [countvectorizer]
pipeline = Pipeline(stages=stages)


In [6]:
pipeline.fit(df).transform(df).show(truncate=False)

+-------------------------------------------+----------------------------------------------+
|terms                                      |features(vocabSize), [index], [term frequency]|
+-------------------------------------------+----------------------------------------------+
|[spark, spark, spark, is, awesome, awesome]|(10,[0,1,3],[3.0,2.0,1.0])                    |
|[I, love, spark, very, very, much]         |(10,[0,2,4,5,6],[1.0,2.0,1.0,1.0,1.0])        |
|[everyone, should, use, spark]             |(10,[0,7,8,9],[1.0,1.0,1.0,1.0])              |
+-------------------------------------------+----------------------------------------------+



Ahora, usemos el StringIndexer() para indexar el corpus y ver si los resultados son consistentes con el método CountVectorizer().

### `FlatMap` para que cada fila tenga un solo término.

In [11]:
from pyspark.sql.types import StringType
df_vocab = df.select('terms').rdd.\
            flatMap(lambda x: x[0]).\
            toDF(schema=StringType()).toDF('terms')
df_vocab.show()

+--------+
|   terms|
+--------+
|   spark|
|   spark|
|   spark|
|      is|
| awesome|
| awesome|
|       I|
|    love|
|   spark|
|    very|
|    very|
|    much|
|everyone|
|  should|
|     use|
|   spark|
+--------+



### Calcular frecuencias de término en el corpus

In [24]:
vocab_freq = df_vocab.rdd.countByValue()
pdf = pd.DataFrame({
        'term': list(vocab_freq.keys()),
        'frequency': list(vocab_freq.values())
    })
pdf
tf = spark.createDataFrame(pdf).orderBy('frequency', ascending=False)
tf.show()

+---------+----------+
|frequency|      term|
+---------+----------+
|        5|   [spark]|
|        2|    [very]|
|        2| [awesome]|
|        1|      [is]|
|        1|       [I]|
|        1|    [love]|
|        1|[everyone]|
|        1|     [use]|
|        1|    [much]|
|        1|  [should]|
+---------+----------+



## Aplicar `StringIndexer()` a df_vocab

In [25]:
from pyspark.ml.feature import StringIndexer
stringindexer = StringIndexer(inputCol='terms', outputCol='StringIndexer(index)')

In [26]:
stringindexer.fit(df_vocab).transform(df_vocab).\
    distinct().\
    orderBy('StringIndexer(index)').show()

+--------+--------------------+
|   terms|StringIndexer(index)|
+--------+--------------------+
|   spark|                 0.0|
| awesome|                 1.0|
|    very|                 2.0|
|      is|                 3.0|
|everyone|                 4.0|
|       I|                 5.0|
|    love|                 6.0|
|  should|                 7.0|
|    much|                 8.0|
|     use|                 9.0|
+--------+--------------------+



El resultado de la indexación es consistente para los tres primeros trimestres. El resto de términos tienen la misma frecuencia que es 1. Estos términos no pueden ser ordenados por frecuencia. Esta podría ser la razón por la que sus índices no coinciden con los resultados del método CountVectorizer().